# Design For User interface using tkinter module

### importing the required libraries and files

In [1]:
from tkinter import *
from tkinter.ttk import *
from tkinter import messagebox
from PIL import Image, ImageDraw, ImageTk
import glob
from os import path
import os
from functools import partial

from identify_and_segment import image_identification, image_segmentation
from numerical import LEVELS
from quiz import QUIZ


Using TensorFlow backend.
42+2
42+2


### defining the class for drawing on canvas using Hand

In [2]:
class PaintCanvas:
    def __init__(self,WINDOW):
        self.WINDOW = WINDOW
        self.old_x = None
        self.old_y = None
        self.penWidth = 10
        self.canvas_width = 400
        self.canvas_height = 150
        self.penColor = 'black'
        self.fg_color = (0,0,0) #for draw 
        self.bg_color = (255,255,255) #for draw
        self.createWidgets()

    def paint(self,e):
        if self.old_x and self.old_y:
            self.c.create_line(self.old_x,self.old_y,e.x,e.y,
                width=self.penWidth,fill=self.penColor,
                capstyle=ROUND,smooth=True
            )
            self.draw.line([self.old_x,self.old_y,e.x,e.y],self.fg_color, width=10)

        self.old_x = e.x
        self.old_y = e.y

    def reset(self,e):    #reseting or cleaning the canvas 
        self.old_x = None
        self.old_y = None 
    
    def createWidgets(self):
        white = (255,255,255)

        x,y = 100, 333
        
        ShowButton(self.WINDOW,"Pen",x,y,self.selectPen)
        ShowButton(self.WINDOW,"Eraser",x+100,y,self.selectEraser)
        ShowButton(self.WINDOW,"Clear",x+200,y,self.selectClear)
        ShowButton(self.WINDOW,"Save",x+300,y,self.saveImage)

        self.c = Canvas(self.WINDOW,width=self.canvas_width,height=self.canvas_height,bg='white')
        self.c.pack()
        self.c.place(x=100,y=365)

        self.c.bind('<B1-Motion>',self.paint)#drwaing the line 
        self.c.bind('<ButtonRelease-1>',self.reset)

        self.image1 = Image.new("RGB", (500, 150), self.bg_color)
        self.draw = ImageDraw.Draw(self.image1)
    
    def selectClear(self):
        self.c.delete("all")
        self.draw.line([0,0,500,500],self.bg_color, width=500)
    
    def selectPen(self):
        self.penWidth = 13
        self.penColor = "black"
        self.fg_color = (0,0,0)
    
    def selectEraser(self):
        self.penWidth = 30
        self.penColor = "white"
        self.fg_color = (255,255,255)
    
    def saveImage(self):
        global QUESTION, ANSWER, level
        Images_path = os.getcwd() + '\\Images\\'
        filename = Images_path+"answer.jpg"

        self.image1.save(filename)
        image_segmentation()
        answer, accuracy = image_identification()
        confirm = messagebox.askquestion('Confirm','Is your number '+str(answer) +' ? (with '+str("{0:.2f}".format(accuracy*100)+'% accuracy).'))
        self.selectClear()
        if confirm == 'no':
            return
        print("predicted number as :",answer)
        
        if int(answer) == int(ANSWER):
            level +=1
            if GAME_TYPE == "NUMERICAL":
                QUESTION  =  LEVELS[level]
                canvas.itemconfigure(label2, text=QUESTION+' =')
                ANSWER = eval(QUESTION.replace('x','*'))
                
            else:
                QUESTION, ANSWER= QUIZ[level]
                canvas.itemconfigure(label2, text=QUESTION)
        else:
            messagebox.showinfo('Wrong','Sorry, your answer is incorrect.\n Try again.')


### defining the drawing objects functions in the interface

In [7]:
def ShowButton(WINDOW,text,X,Y,Command):
    panel = Button(WINDOW, text=text,style='TButton',command=Command)
    panel.pack()
    panel.place(x=X,y=Y)

def menu():
    global canvas1
    canvas1 = Canvas(WINDOW, width=600, height=600)
    canvas1.pack()
    
    img = Image.open((bg2_img))
    img = img.resize((600, 600), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    bg_label = canvas1.create_image((0,0), image=img, anchor=N+W)
    canvas1.image = img 
    label1 = canvas1.create_text((300,150), text="     HandWritten Digit    \n Identification and QUIZ",                        font="MSGothic 30 bold", fill="green")


    Math_btn = Button(WINDOW,text='Math',style='W.TButton',command=partial(main,'NUMERICAL'))
    Math_button = canvas1.create_window(300,300,window=Math_btn)

    Quiz_btn = Button(WINDOW,text='Quiz',style='W.TButton',command=partial(main,'QUIZ'))
    Quiz_button = canvas1.create_window(300,350,window=Quiz_btn)

    Exit_btn = Button(WINDOW,text='Exit',style='W.TButton',command=quit)
    Exit_button = canvas1.create_window(300,400,window=Exit_btn)

def main(type):
    global canvas, QUESTION , ANSWER , GAME_TYPE , label2, level
    canvas1.delete('all')
    img = Image.open((bg_img))
    img = img.resize((600, 600), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    canvas = Canvas(WINDOW, height=600, width=600)
    canvas.pack()
    canvas.place(x=0,y=0)

    bg_label = canvas.create_image((0,0), image=img, anchor=N+W)
    canvas.image = img 
    label1 = canvas.create_text((300,100), text="Write Your Answer", font="MSGothic 40 bold", fill="#652828")
    GAME_TYPE = type
    level = 0
    if GAME_TYPE == "NUMERICAL":
        QUESTION  =  LEVELS[level]
        ANSWER = eval(QUESTION)
        label2 = canvas.create_text((300,225), text= QUESTION+' =', font="MSGothic 75 bold", fill="#652828")
    else:
        QUESTION, ANSWER= QUIZ[level]
        label2 = canvas.create_text((300,225), text= QUESTION, font="MSGothic 18 bold", fill="#652828")

    PaintCanvas(WINDOW)

### defining global variables and Showing the Main interface

In [8]:

Images_path = os.getcwd() + '\\Images\\'
bg_img = Images_path + "background.jpg"
bg2_img = Images_path + "bg.jpg"

WINDOW = Tk()
WINDOW.geometry('+200+50') #show WINDOW starting from x-200 and y-50
WINDOW.geometry('600x600') #set the geomentry of WINDOW
WINDOW.title("Machine learning")
style = Style()  
style.configure('W.TButton', font =('calibri', 15, 'bold', 'underline'), 
                foreground = 'green', borderwidth = '4') 
style.configure('TButton', font =('calibri', 13, 'bold', 'underline'), 
                foreground = 'green', borderwidth = '4') 
menu()
WINDOW.mainloop()

Number of contours = 10
Format of hieraechy = [[Next, Previous, First_Child, Parent]]
Number of countours after filtering =  2
Number of countours after filtering =  2


# Image Segmentation and Identification Part

### importing the required libraries

In [20]:
#impoting packages
import imageio
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import os
import cv2

### Loading the save model and defining the paths

In [21]:
# load the model
from keras.models import load_model
model = load_model("test_model.h5")

#paths to images
Images_path = os.getcwd() + '\\Images\\'
non_segmented_image_path = Images_path + 'answer.jpg'
imageLocation = Images_path + 'Segmented_Images\\'


## 1. Image Segmentation Part

#### finding countours and hierarchy using openCV

In [22]:
img = cv2.imread(non_segmented_image_path)

#segmenting the image with counter from openCV
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(imgray, 127, 255, 0)
contours, hierarchies = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print("Number of contours = " + str(len(contours)))
print("Format of hieraechy = [[Next, Previous, First_Child, Parent]]")


Number of contours = 14
Format of hieraechy = [[Next, Previous, First_Child, Parent]]


### filtering the unwanted contours

In [23]:
cv2.drawContours(img, contours, -1, (0, 255, 0), 1)
#cv2.imshow("img",img)

#deleting the images already in the directory
imageLocation = Images_path + 'Segmented_Images\\'
for file in os.listdir(imageLocation):
        os.remove(imageLocation+file) 

#sorted_ctrs = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0]  )

#removing unnecessary contours        
#hierarchy[0,i,3] #hierarchy=[[Next, Previous, First_Child, Parent]]
contours_list = []

for contour,hierarchy in zip(contours,hierarchies[0]):
    #print(hierarchy)
    if hierarchy[3] == 0 or cv2.contourArea(contour) > 1000 and hierarchy[2] != -1 :
    #if cv2.contourArea(contour) > 100 or hierarchy[3] == 0:
        contours_list.append(contour)
        


### sorting the contours in order

In [24]:
#sorting contours
contours_list = sorted(contours_list, key=lambda ctr: cv2.boundingRect(ctr)[0]  )
print('Number of countours after filtering = ',len(contours_list[1:]) )        


Number of countours after filtering =  2


### saving each of the contours as an image

In [28]:
#saving and naming Contours in images
for i,contour in enumerate(contours_list[1:]):
        (x, y, w, h) = cv2.boundingRect(contour)

        #padding images and cropping
        p = 10  
        x,y,w,h = x-p,y-p,w+p+10,h+p+10
        roi = img[y:y+h, x:x+w] #roi = region of image

        #saving the image
        name = imageLocation + 'roi'+str(i)+'.png'
        cv2.imwrite(name, roi)

        #saving image with aspect-ratio
        img2 = Image.open(name)
        w1,h1 = img2.size
        background = Image.open(Images_path + 'number_background.png')
        w2,h2 = background.size
        pasteLocation = int(w2/2) - int(w1/2), int(h2/2) - int(h1/2)
        background.paste(img2,pasteLocation)
        background.save(name,"PNG")

## 2. Image Segmentation Part

### defining variable for adding the predicted numbers

In [32]:
predicted_number = []
prediction_accuracy = 0
No_of_images = 0

### identifying each saved images using the model

In [33]:
#loop though all the images in the saved folder
for image in os.listdir(imageLocation):
    No_of_images += 1
    im = imageio.imread(imageLocation+image)
    #getting image
    #im = imageio.imread(imageLocation+'roi2.png')
    dim = (28,28)

    #preprocessing image
    im = cv2.resize(im, dim, interpolation = cv2.INTER_AREA)
    #plt.imshow(im)
    gray = np.dot(im[...,:3], [0.299, 0.587, 0.114])
    gray = gray.reshape(1, 28, 28, 1)
    gray = 255-gray
    gray = gray/255

    # predicting the image
    prediction = model.predict(gray)
    predicted_number.append(str(prediction.argmax()))
    prediction_accuracy += max(prediction[0])

#converting the all predicted digit to single string 
answer = ''.join(predicted_number)
accuracy = 0

#only if images is found
if No_of_images != 0:
    accuracy = prediction_accuracy/No_of_images

    print('The predicted number is {} with {}% accuracy'.format(answer , accuracy))

The predicted number is 10 with 1.0% accuracy
